A big part of time series analysis involves filtering - i.e. changing attributes of a time series or deconstructing it into its component parts. Often we need to do quite a bit of time series before we build a model to simulate the underlying process.


To run the ADF test we need to chose a lag length so that the residuals aren't serially correlated (aka Autocorrelation), this is to minimize error terms in a time series as to avoid transferring them from one period to another. For choosing the lags the AIC (Akaike's information criterion) will be minimized. 

The behavior of time series models, the constant c has an important effect on the long-long term forecasts obtained from these models. 

    - If c = 0 and d = , the long-term forecase

After graphing the difference for a stationary model determining ARMA, AR or MA 

The special cases of ARIMA models 

    White noise             ARIMA(0,0,0)
    Random walk             ARIMA(0,1,0) with no constant 
    Random walk with drift  ARIMA(0,1,0) with a constant
    Autoregression          ARIMA(p,0,0)
    Moving average          ARIMA(0,0,q)

ACF and PACF cannot be used to choose the order of a model when both the orders q and p are non-zero. Instead there are other models the AIC and BIC.

    a. The model with the lower AIC Score makes better predictions 
    

If you receive a value error this is a bad model for the data. 

In [1]:
#imports
import pandas as pd 
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
import pickle
import datetime
import seaborn as sns
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from scipy import stats

In [6]:
# load the files
pbc_afa = pd.read_csv('../data/pre_bc_afa.csv', index_col=0)
pbc_afr = pd.read_csv('../data/pre_bc_afr.csv', index_col=0)
pbc_asr = pd.read_csv('../data/pre_bc_asr.csv', index_col=0)
pbc_asa = pd.read_csv('../data/pre_bc_asa.csv', index_col=0)
pbc_ea = pd.read_csv('../data/pre_bc_ea.csv', index_col=0)
pbc_er = pd.read_csv('../data/pre_bc_er.csv', index_col=0)
pbc_nr = pd.read_csv('../data/pre_bc_nr.csv', index_col=0)
pbc_na = pd.read_csv('../data/pre_bc_na.csv', index_col=0)
pbc_sr = pd.read_csv('../data/pre_bc_sr.csv', index_col=0)
pbc_sa = pd.read_csv('../data/pre_bc_sa.csv', index_col=0)

In [3]:
# load the models
import warnings
afa = pickle.load(open('../models/af_act_model.sav', 'rb'))
afr = pickle.load(open('../models/af_rec_model.sav', 'rb')) 
asa = pickle.load(open('../models/asia_act_model.sav', 'rb'))
asr = pickle.load(open('../models/asia_rec_model.sav', 'rb')) 
ea = pickle.load(open('../models/eu_act_model.sav', 'rb')) 
er = pickle.load(open('../models/eu_rec_model.sav', 'rb')) 
na = pickle.load(open('../models/noam_act_model.sav', 'rb')) 
nr = pickle.load(open('../models/noam_rec_model.sav', 'rb'))
sa = pickle.load(open('../models/soam_act_model.sav', 'rb')) 
sr = pickle.load(open('../models/soam_rec_model.sav', 'rb'))

Africa Active Summary

In [ ]:
def evaluate_arima_model(data, arima_order):
    # Needs to be an integer because it is later used as an index.
    # Use int()
    split=int(len(data) * 0.7) 
    # Make train and test variables, with 'train, test'
    train, test = data[0:split], data[split:len(data)]
    past=[x for x in train]
    # make predictions. Declare a variable with that name
    predictions = list()
    for i in range(len(test)):#timestep-wise comparison between test data and one-step prediction ARIMA model. 
        past_bc, fitted_lambda = stats.boxcox(past)
        test_bc = stats.boxcox(test, fitted_lambda)
        model = ARIMA(past_bc, order=arima_order)
        model_fit = model.fit(disp=0)
        future = model_fit.forecast()[0]
        # Append() here
        predictions.append(future)
        past.append(test_bc[i])
    # calculate out of sample error
    error = mean_squared_error(test_bc, predictions)
    # Return the error
    return error

In [ ]:
error = evaluate_arima_model(pbc_afa, 0,0,0)

In [ ]:
# def evaluate_arima_model(data, arima_order):
#     # Needs to be an integer because it is later used as an index.
#     # Use int()
#     split=int(len(data) * 0.7) 
#     # Make train and test variables, with 'train, test'
#     train, test = data[0:split], data[split:len(data)]
#     past=[x for x in train]
#     # make predictions. Declare a variable with that name
#     predictions = list()
#     for i in range(len(test)):#timestep-wise comparison between test data and one-step prediction ARIMA model. 
#         past_bc, fitted_lambda = stats.boxcox(past)
#         test_bc = stats.boxcox(test, fitted_lambda)
#         model = ARIMA(past_bc, order=arima_order)
#         model_fit = model.fit(disp=0)
#         future = model_fit.forecast()[0]
#         # Append() here
#         predictions.append(future)
#         past.append(test_bc[i])
        
#         return past, predictions

In [27]:
pbc_afa

,active
date,
2020-03-02,1.0
2020-03-03,1.0
2020-03-04,1.0
2020-03-05,2.0
2020-03-06,2.0
...,...
2020-12-27,26779.0
2020-12-28,26182.0
2020-12-29,26446.0


In [25]:
split=int(len(pbc_afa) * 0.7)
train, test = pbc_afa[0:split], pbc_afa[split:len(pbc_afa)]
print(train.shape, test.shape)


(213, 1) (92, 1)


In [23]:
t = train[train > 0].sum()
type(train)

pandas.core.frame.DataFrame

In [26]:
bc_train, fitted_lambda = stats.boxcox(train)

ValueError: Data must be 1-dimensional.

In [ ]:
for i in range(len(test)):
    past_bc, fitted_lambda = stats.boxcox(past)
    test_bc = stats.boxcox(test, fitted_lambda)
    model = ARIMA(past_bc, order=arima_order)
    model_fit = model.fit(disp=0)
    future = model_fit.forecast()[0]
    # Append() here
    predictions.append(future)
    past.append(test_bc[i])
    

In [ ]:
pbc_afa

In [ ]:
past, predictions = evaluate_arima_model(pbc_afa, (0,0,0))

In [28]:
afa.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:                 active   No. Observations:                  305
Model:                     ARMA(0, 0)   Log Likelihood               -3371.179
Method:                           css   S.D. of innovations          15277.046
Date:                Thu, 09 Sep 2021   AIC                           6746.358
Time:                        11:36:27   BIC                           6753.798
Sample:                    03-02-2020   HQIC                          6749.334
                         - 12-31-2020                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.436e+04    874.761     16.417      0.000    1.26e+04    1.61e+04
==============================================================================
"""

Africa Recover Summary

In [29]:
afr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              ARMA Model Results                              
==============================================================================
Dep. Variable:              recovered   No. Observations:                  294
Model:                     ARMA(0, 0)   Log Likelihood               -3858.533
Method:                           css   S.D. of innovations         121215.320
Date:                Thu, 09 Sep 2021   AIC                           7721.066
Time:                        11:36:30   BIC                           7728.433
Sample:                    03-13-2020   HQIC                          7724.017
                         - 12-31-2020                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       9.118e+04   7069.421     12.897      0.000    7.73e+04    1.05e+05
==============================================================================
"""

Asia Active Summary 

In [ ]:
asa.summary()

Asia Recovered Summary

In [ ]:
asr.summary()

Europe Active Summary 

In [ ]:
ea.summary()

In [ ]:
Europe Recovery Summary 

In [ ]:
er.summary()

In [ ]:
North America Active Summary

In [ ]:
na.summary()

In [ ]:
North America Recovery Summary 

In [ ]:
nr.summary()

South America Active Summary 

In [ ]:
sa.summary()

In [ ]:
South America Recovery Summary 

In [ ]:
sr.summary()

When fitting start_params, residuals are obtained from an AR fit, then an ARMA(p,q) model is fit via OLS using these residuals. If start_ar_lags is None, fit an AR process according to best BIC. If start_ar_lags is not None, fits an AR process with a lag length equal to start_ar_lags. See ARMA._fit_start_params_hr for more information.

AR models have theoretical PACFs with non-zero values at the AR terms in the model and zero values elsewhere. The ACF will taper to zero in some fashion.

MA models have theoretical ACFs with non-zero values at the MA terms in the model and zero values elsewhere.  The PACF will taper to zero in some fashion.

If the series autocorrelations are non-significant, then the series is random (white noise; the ordering matters, but the data are independent and identically distributed.) You’re done at that point.

If first differences were necessary and all the differenced autocorrelations are non-significant, then the original series is called a random walk and you are done. (A possible model for a random walk is $ x_t = \delta + x_{t-1} + w_t $ . The data are dependent and are not identically distributed; in fact both the mean and variance are increasing through time.)

In [ ]:
def transf(data, model):
    # Split data
    split=int(len(data) * 0.7) 
    # Make train and test variables, train, test
    train, test = data[0:split], data[split:len(data)]
    # Fit the data to the model      
    model_fit = model.fit()
    # Forecast the data 
    forecast = model_fit.forecast(24)
    
    return forecast, test

In [ ]:
# Africa, Asia.recovered ARIMA(0,0,0) white noise
# Africa Recovered BIC = 7728.433
# Africa Active BIC = 6753.798
# Asia Recovered BIC = 11626.831
a_a_models = [asr, afa, afr]
a_a_sample_data = [pbc_afa, pbc_afr, pbc_asr]

In [ ]:
# Asia active, South America Recovered, ARIMA(0,0,1)
# Asia Active BIC = 9692.586
# South America Active BIC = 
# South America Recovered BIC =
# The PACF will taper to zero in some fashion.

a_r_models = [asa, sr]
a_r_samples_data = [pbc_asa, pbc_sr]

In [ ]:
# Europe recovered ARIMA(2,0,0)
# Fit two lag start length
# An AR(2) has two spikes in the PACF and a sinusoidal ACF that converges to 0.
er
pbc_er

In [ ]:
# Europe active, North America, South America Active ARIMA(1,0,0)
# Fit one lag start length
# An AR(1) model has a single spike in the PACF and an ACF with a pattern
n_s_e_models = [na, nr, sa, ea]
sample_data = [pbc_ea, pbc_nr, pbc_na, pbc_sa] 


for i in sample_data: 
    train, test = split_data(i)
    for ii in model_list:
        
    

In [ ]:
#North America Active 
na_pred = pd.DataFrame(na.predict(), columns=['active'])

#North America Recovered
nr_pred = pd.DataFrame(nr.predict(), columns=['recovered'])

#South America Active
sa_pred = pd.DataFrame(sa.predict(), columns=['active'])

#South America Recovered
sr_pred = pd.DataFrame(sr.predict(), columns=['recovered'])

#Europe Active
ea_pred = pd.DataFrame(ea.predict(), columns=['active'])

#Europe Recovered 
er_pred = pd.DataFrame(er.predict(), columns=['recovered'])

#Asia Active
asa_pred = pd.DataFrame(asa.predict(), columns=['active'])

#Asia Recovered
asr_pred = pd.DataFrame(asr.predict(), columns=['recovered'])

#Africa Active
afa_pred = pd.DataFrame(afa.predict(), columns=['active'])

#Africa Recovered
afr_pred = pd.DataFrame(afr.predict(), columns=['recovered'])

Forecasting refers to the process of analyzing and elucidating a future state. This process takes the past and the current information into account in a bid to predict facts for future events. This is a quantitative analysis of the data. A forecast is more accurate compared to a prediction. This is because forecast are derived by analyzing a set of past data from past and present trends. The analysis helps in coming up with a model that is scientifically backed and the probability of it being wrong are minimal. Prediction is a statement which explains one possible outcome.


In [ ]:


fig, axes = plt.subplots(5, 2, figsize=(14, 14))

# Define the date format
date_form = DateFormatter("%m-%d")

# plotting subplots
sns.pointplot(ax=axes[0,0], x=na_pred.index, y='active', data=na_pred, color='b')
sns.pointplot(ax=axes[0,0], x=pbc_na.index, y='active', data=pbc_na, color='r')
axes[0, 0].set_title('North America Model Comparison', loc='left', pad=15)
axes[0, 0].margins(x=0)
axes[0, 0].xaxis.set_major_formatter(date_form)
axes[0, 0].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[0,1], x=nr_pred.index, y='recovered', data=nr_pred, color='b')
sns.pointplot(ax=axes[0,1], x=pbc_nr.index, y='recovered', data=pbc_nr, color='r')
axes[0, 1].margins(x=0)
axes[0, 1].xaxis.set_major_formatter(date_form)
axes[0, 1].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[1,0], x=asa_pred.index, y='active', data=asa_pred, color='b')
sns.pointplot(ax=axes[1,0], x=pbc_asa.index, y='active', data=pbc_asa, color='r')
axes[1, 0].set_title('Asia Model Comparison', loc='left', pad=15)
axes[1, 0].margins(x=0)
axes[1, 0].xaxis.set_major_formatter(date_form)
axes[1, 0].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[1,1], x=asr_pred.index, y='recovered', data=asr_pred, color='b')
sns.pointplot(ax=axes[1,1], x=pbc_asr.index, y='recovered', data=pbc_asr, color='r')
axes[1, 1].margins(x=0)
axes[1, 1].xaxis.set_major_formatter(date_form)
axes[1, 1].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[2,0], x=ea_pred.index, y='active', data=ea_pred, color='b')
sns.pointplot(ax=axes[2,0], x=pbc_ea.index, y='active', data=pbc_ea, color='r')
axes[2, 0].set_title('Europe Model Comparison', loc='left', pad=15)
axes[2, 0].margins(x=0)
axes[2, 0].xaxis.set_major_formatter(date_form)
axes[2, 0].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[2,1], x=er_pred.index, y='recovered', data=er_pred, color='b')
sns.pointplot(ax=axes[2,1], x=pbc_er.index, y='recovered', data=pbc_er, color='r')
axes[2, 1].margins(x=0)
axes[2, 1].xaxis.set_major_formatter(date_form)
axes[2, 1].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[3,0], x=afa_pred.index, y='active', data=afa_pred, color='b')
sns.pointplot(ax=axes[3,0], x=pbc_afa.index, y='active', data=pbc_afa, color='r')
axes[3, 0].set_title('Africa Model Comparison', loc='left', pad=15)
axes[3, 0].margins(x=0)
axes[3, 0].xaxis.set_major_formatter(date_form)
axes[3, 0].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[3,1], x=afr_pred.index, y='recovered', data=afr_pred, color='b')
sns.pointplot(ax=axes[3,1], x=pbc_afr.index, y='recovered', data=pbc_afr, color='r')
axes[3, 1].margins(x=0)
axes[3, 1].xaxis.set_major_formatter(date_form)
axes[3, 1].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[4,0], x=sa_pred.index, y='active', data=sa_pred, color='b')
sns.pointplot(ax=axes[4,0], x=pbc_sa.index, y='active', data=pbc_sa, color='r')
axes[4, 0].set_title('South America Model Comparison', loc='left', pad=15)
axes[4, 0].margins(x=0)
axes[4, 0].xaxis.set_major_formatter(date_form)
axes[4, 0].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))
sns.pointplot(ax=axes[4,1], x=sr_pred.index, y='recovered', data=sr_pred, color='b')
sns.pointplot(ax=axes[4,1], x=pbc_sr.index, y='recovered', data=pbc_sr, color='r')
axes[4, 1].margins(x=0)
axes[4, 1].xaxis.set_major_formatter(date_form)
axes[4, 1].xaxis.set_major_locator(mdates.WeekdayLocator(interval=5))


# automatically adjust padding horizontally
# as well as vertically.
plt.tight_layout()
 
# display plot
plt.show()

Africa, Asia.recovered ARIMA(0,0,0)
Asia active, South America Recovered, ARIMA(0,0,1)
Europe recovered ARIMA(2,0,0)
Europe active, North America, South America Active ARIMA(1,0,0)

In [ ]:
def show_future(model, df, s):
    forecast = model.forecast(12)
    forcast_period = 6
    
    #Convert dataset to a month daterange
    date_range = pd.date_range(df.index[-1],
                              periods = forcast_period,
                              freq='MS').strftime("%Y-%m-%d").tolist()
    
    #Convert months to date-time object
    future_months = pd.DataFrame(date_range, columns = ['month'])
    future_months['month'] = pd.to_datetime(future_months['month'], format='%Y-%m-%d')
    future_months.set_index('month', inplace = True)
    
    #Create prediction column using the forcast
    if s == 1:
        future_months['active'] = pd.Series(dict(zip(date_range,(forecast[0]))),dtype='object')
#         future_months['month'] = pd.to_datetime(future_months['month'], format='%Y-%m-%d').dt.date
    if s == 2:
        future_months['recovered'] = pd.Series(dict(zip(date_range,(forecast[0]))),dtype='object')

    #Append future predictions
    past_future = pd.concat([df,future_months])
    past_future.index = pd.to_datetime(past_future.index)

    return past_future

In [ ]:
a = show_future(afa, pbc_afa, 1)
a
# sns.lineplot(data=a)

In [ ]:
# Declare a variable called forecast_period with the amount of months to forecast, and
# create a range of future dates that is the length of the periods you've chosen to forecast
forecast_period = 12

# Convert that range into a dataframe that includes your predictions
date_range = pd.date_range(y_log.index[-1], 
                           periods = forecast_period,
                           freq='MS').strftime("%Y-%m-%d").tolist()
print(date_range)
# Plot your future predictions
future_months = pd.DataFrame(date_range, columns = ['Month'])
future_months['Month'] = pd.to_datetime(future_months['Month'])
future_months.set_index('Month', inplace = True)

future_months['Prediction'] = pd.Series(dict(zip(date_range,(forecast[0]))),dtype='object')

# Plot your future predictions
plt.figure(figsize=(15,10))
plt.plot(y_log)
plt.plot(y_log[].append(future_months['Prediction']))
plt.show()

In [ ]:
pbc_na.iloc[-1]